In [ ]:
# | default_exp _testing.local_kafka_broker

In [ ]:
# | export
from unittest.mock import MagicMock

from aiokafka import AIOKafkaConsumer, AIOKafkaProducer

from fastkafka._components.meta import patch
from fastkafka._components.logger import get_logger

In [ ]:
# | export

logger = get_logger(__name__)

# Local Kafka broker
> In-memory mockup of Kafka broker protocol

In [ ]:
class LocalKafkaBroker():
    def __init__():
        pass
    
    def connect():
        pass
    
    def dissconnect():
        pass
    
    def produce():
        pass
    
    def consume():
        pass

## Consumer patching

We need to patch AIOKafkaConsumer methods so that we can redirect the consumer to our local kafka broker.

Patched methods:

- [ ] \_\_init\_\_
- [ ] start
- [ ] subscribe
- [ ] stop
- [ ] getmany

Patching \_\_init\_\_ so that we pass the LocalKafkaBroker instance to all instances of AIOKafkaConsumer in the patched namespace

In [ ]:
# | export

def _patch_AIOKafkaConsumer_init(broker: LocalKafkaBroker) -> None:
    @patch
    def __init__(self: AIOKafkaConsumer, broker: LocalKafkaBroker = broker) -> None:
        logger.info("AIOKafkaConsumer patched __init__() called()")
        self.broker = broker

In [ ]:
with MagicMock() as broker_mock:
    _patch_AIOKafkaConsumer_init(broker_mock)
    consumer = AIOKafkaConsumer()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()


In [ ]:
# | export

def _patch_AIOKafkaConsumer_start() -> None:
    @patch
    async def start(self: AIOKafkaConsumer) -> None:
        logger.info("AIOKafkaConsumer patched start() called()")

In [ ]:
with MagicMock() as broker_mock:
    _patch_AIOKafkaConsumer_init(broker_mock)
    consumer = AIOKafkaConsumer()
    
    _patch_AIOKafkaConsumer_start()
    await consumer.start()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()


In [ ]:
# | export

def _patch_AIOKafkaConsumer_subscribe() -> None:
    @patch
    def subscribe(self: AIOKafkaConsumer) -> None:
        logger.info("AIOKafkaConsumer patched subscribe() called!")

In [ ]:
with MagicMock() as broker_mock:
    _patch_AIOKafkaConsumer_init(broker_mock)
    consumer = AIOKafkaConsumer()
    
    _patch_AIOKafkaConsumer_subscribe()
    consumer.subscribe()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called!


In [ ]:
# | export

def _patch_AIOKafkaConsumer_stop() -> None:
    @patch
    async def stop(self: AIOKafkaConsumer) -> None:
        logger.info("AIOKafkaConsumer patched stop() called!")

In [ ]:
with MagicMock() as broker_mock:
    _patch_AIOKafkaConsumer_init(broker_mock)
    consumer = AIOKafkaConsumer()
    
    _patch_AIOKafkaConsumer_stop()
    await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched stop() called!


In [ ]:
# | export

def _patch_AIOKafkaConsumer_getmany() -> None:
    @patch
    async def getmany(self: AIOKafkaConsumer) -> None:
        logger.info("AIOKafkaConsumer patched getmany() called!")

In [ ]:
with MagicMock() as broker_mock:
    _patch_AIOKafkaConsumer_init(broker_mock)
    consumer = AIOKafkaConsumer()
    
    _patch_AIOKafkaConsumer_getmany()
    await consumer.getmany()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched getmany() called!


In [ ]:
# | export

def _patch_AIOKafkaConsumer(broker: LocalKafkaBroker) -> None:
    _patch_AIOKafkaConsumer_init(broker)
    _patch_AIOKafkaConsumer_start()
    _patch_AIOKafkaConsumer_subscribe()
    _patch_AIOKafkaConsumer_stop()
    _patch_AIOKafkaConsumer_getmany()

In [ ]:
with MagicMock() as broker_mock:
    _patch_AIOKafkaConsumer(broker_mock)

## Producer patching

We need to patch AIOKafkaProducer methods so that we can redirect the producer to our local kafka broker

- [ ] \_\_init\_\_
- [ ] start
- [ ] stop
- [ ] send

In [ ]:
# | export

def _patch_AIOKafkaProducer_init(broker: LocalKafkaBroker) -> None:
    @patch
    def __init__(self: AIOKafkaProducer, broker: LocalKafkaBroker = broker) -> None:
        logger.info("AIOKafkaProducer patched __init__() called()")
        self.broker = broker

In [ ]:
with MagicMock() as mock_broker:
    _patch_AIOKafkaProducer_init(broker=mock_broker)
    producer = AIOKafkaProducer()

[INFO] __main__: AIOKafkaProducer patched __init__() called()


In [ ]:
# | export

def _patch_AIOKafkaProducer_start() -> None:
    @patch
    async def start(self: AIOKafkaProducer) -> None:
        logger.info("AIOKafkaProducer patched start() called()")

In [ ]:
with MagicMock() as mock_broker:
    _patch_AIOKafkaProducer_init(broker=mock_broker)
    producer = AIOKafkaProducer()
    
    _patch_AIOKafkaProducer_start()
    await producer.start()

[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched start() called()


In [ ]:
# | export

def _patch_AIOKafkaProducer_stop() -> None:
    @patch
    async def stop(self: AIOKafkaProducer) -> None:
        logger.info("AIOKafkaProducer patched stop() called()")

In [ ]:
with MagicMock() as mock_broker:
    _patch_AIOKafkaProducer_init(broker=mock_broker)
    producer = AIOKafkaProducer()

    _patch_AIOKafkaProducer_stop()
    await producer.stop()

[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched stop() called()


In [ ]:
# | export

def _patch_AIOKafkaProducer_send() -> None:
    @patch
    async def send(self: AIOKafkaProducer) -> None:
        logger.info("AIOKafkaProducer patched send() called()")

In [ ]:
with MagicMock() as mock_broker:
    _patch_AIOKafkaProducer_init(broker=mock_broker)
    producer = AIOKafkaProducer()

    _patch_AIOKafkaProducer_send()
    await producer.send()

[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched send() called()


In [ ]:
# | export

def _patch_AIOKafkaProducer(broker: LocalKafkaBroker) -> None:
    _patch_AIOKafkaProducer_init(broker)
    _patch_AIOKafkaProducer_start()
    _patch_AIOKafkaProducer_stop()
    _patch_AIOKafkaProducer_send()

In [ ]:
with MagicMock() as mock_broker:
    _patch_AIOKafkaProducer(mock_broker)